In [1]:
# # Installing all the dependencies:

!pip install langchain langchain-community PyPDF2 huggingface_hub sentence_transformers chromadb chroma gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB

In [2]:
# # Importing all necessary libraries:

from PyPDF2 import PdfReader
import PyPDF2
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
import getpass
import os
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import gradio as gr
import time
from google.colab import drive
from langchain.schema import Document

In [3]:
# Give the access tokens of Huggingface to access models:

if "HUGGINGFACEHUB_API_TOKEN" not in os.environ:
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Provide your HUGGINGFACEHUB TOKEN")

Provide your HUGGINGFACEHUB TOKEN··········


In [4]:
# Access the file from gdrive:

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Quantiphi/ConceptsofBiology.pdf'

Mounted at /content/drive


In [5]:
# As per instruction picking up 2 chapters from PDF i.e till page 54:

def read_first_n_pages(file_path, n):

    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # Check the number of pages in the PDF
        total_pages = len(reader.pages)
        print(f'Total pages in the PDF: {total_pages}')

        # Determine the number of pages to read
        pages_to_read = min(total_pages, n)

        # Extract text from the first n pages
        text = ''
        for i in range(pages_to_read):
            page = reader.pages[i]
            text += page.extract_text() + '\n'

        return text

# Read only 2 chapters having 54 pages:
raw_text = read_first_n_pages(file_path, 54)

Total pages in the PDF: 615


In [6]:
raw_text

'\n\n\t\n\t      Concepts of Biology           SENIOR CONTRIBUTING AUTHORS SAMANTHA FOWLER, CLAYTON STATE UNIVERSITY REBECCA ROUSH, SANDHILLS COMMUNITY COLLEGE JAMES WISE, HAMPTON UNIVERSITY          \n\nOpenStax  \nRice University  \n6100 Main Street MS- 375 \nHouston, Texas 77005  \nTo learn more about OpenStax, visit http s://openstax.org.  \nIndividual print copies and bulk orders can be purchased through our website.  \n©2023 Rice University.  Textbook content produced by OpenStax i s licensed under a Creative Commons \nAttribution 4.0 International License  (CC BY 4.0) . Under this license, any user of this textbook or the textbook \ncontents herein must provide proper attribution as follows:   \n-If you redistribute this textbook in a digita l format (including but not limited to  PDF and HTML), then y ou\nmust retain on every page the following attribution:\n“Access for free at openstax.org .”\n-If you redistribute this textbook in a print format, then you must include on every

Chunking the text to store it in vector DB.

In [ ]:
# # Approach 1: Sliding window technique to split the text.

# text_splitter = CharacterTextSplitter(
#     separator = "\n",
#     chunk_size = 2000,
#     chunk_overlap  = 300,
#     length_function = len,
# )
# texts = text_splitter.split_text(raw_text)

In [7]:
# # Approach 2: Adaptive chunking technique to split the text.

from transformers import GPT2Tokenizer

def adaptive_chunking(text, max_tokens, tokenizer):
    sentences = text.split('. ')
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        # Tokenize the sentence to get its length
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        sentence_length = len(sentence_tokens)

        # If adding the sentence exceeds the max length, finalize the current chunk
        if current_length + sentence_length > max_tokens:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_length = 0

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_length += sentence_length

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
max_tokens = 100  # Set the maximum number of tokens per chunk

chunks = adaptive_chunking(raw_text, max_tokens, tokenizer)
# for i, chunk in enumerate(chunks):
#     print(f"Chunk {i + 1}:\n{chunk}\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2430 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
len(chunks)

399

In [10]:
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
## Loading the model from Huggingface:

llm=HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.3")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [12]:
## Using Chroma DB to store the data in VectorDB:

documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore_db = Chroma.from_documents(documents, embeddings)
retriever = vectorstore_db.as_retriever()

In [14]:
## Creating a chain of retriever, prompt, llm, structured output:

rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [19]:
## Testing the response:

response = rag_chain.invoke("the smallest unit of biological structure that meets the functional requirements of “living” is the .......")
response

'Answer: cell'

In [20]:
## Using gradio to interact with our RAG pipeline:

def process_question(user_question):
    start_time = time.time()

    # Directly using the user's question as input for rag_chain.invoke
    response= rag_chain.invoke(user_question)

    # Measure the response time
    end_time = time.time()
    response_time = f"Response time: {end_time - start_time:.2f} seconds."

    # Combine the response and the response time into a single string
    full_response = f"{response}\n\n{response_time}"

    return full_response

# Setup the Gradio interface
iface = gr.Interface(fn=process_question,
                     inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
                     outputs=gr.Textbox(),
                     title="Quantiphi Chatbot",
                     description="Ask any question about your document, and get an answer along with the response time.")

# Launch the interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f7af1611dc96483aa9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
